## Load Base Stable Diffusion Model

In [1]:
from diffusers import StableDiffusionImg2ImgPipeline
from pruna.algorithms.smasher_config import AutoSmasherConfig
from pruna.smash import smash
import torch
import requests
from PIL import Image
from io import BytesIO

from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.unet.load_attn_procs('hunz/stable-diffusion-xl-lora')

pipe.to("cuda")

prompt = "Iron Man flying full speed with blue flames"

Post-training Optimization Tool is deprecated and will be removed in the future. Please use Neural Network Compression Framework instead: https://github.com/openvinotoolkit/nncf
Nevergrad package could not be imported. If you are planning to use any hyperparameter optimization algo, consider installing it using pip. This implies advanced usage of the tool. Note that nevergrad is compatible only with Python 3.7+


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

## Smash it!

In [2]:
from pruna.algorithms.smasher_config import SmasherConfig

smasher_config = SmasherConfig()
smasher_config['compiler'] = 'diffusers'
smasher_config['max_batch_size'] = 1
smasher_config['image_height'] = 1024
smasher_config['image_width'] = 1024
smasher_config['version'] = 'xl-1.0'
smasher_config['scheduler'] = "DDIM"

In [3]:
smashed_model = smash(
        model=pipe,
        data_module="LAION256",
        api_key='your-api-key',
        model_config=None,
        smasher_config=smasher_config,
        device='cuda',
    )

[ WARNING ] Found cached dataset parquet (/nfs/homedirs/rachwan/.cache/huggingface/datasets/nannullna___parquet/nannullna--laion_subset-23ff82ebc89d23ff/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

Compile...
Exporting model: /nfs/students/rachwan/huggingface/.cache/onnx/clip/model.onnx
Generating optimizing model: /nfs/students/rachwan/huggingface/.cache/onnx/clip.opt/model.onnx


2023-11-14 17:36:55.796546480 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 1231016, index: 1, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-11-14 17:36:55.796668178 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 1231018, index: 3, mask: {2, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-11-14 17:36:55.796698488 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 1231017, index: 2, mask: {65, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-11-14 17:36:55.796918325 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 1231019, index: 4, mask: {66, }, error code: 22 error msg: Invalid argument. Specify the

[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[W] Inference failed. You may want to try enabling partitioning to see better results. Note: Error was:
[ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Failed to load model with error: /onnxruntime_src/onnxruntime/core/graph/model.cc:146 onnxruntime::Model::Model(onnx::ModelProto&&, const PathString&, const IOnnxRuntimeOpSchemaRegistryList*, const onnxruntime::logging::Logger&, const onnxruntime::ModelOptions&) Unsupported model IR version: 9, max supported IR version: 8


...

[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[W] Inference failed. You may want to try enabling partitioning to see better results. Note: Error was:
[ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Failed to load model with error: /onnxruntime_src/onnxruntime/core/graph/model.cc:146 onnxruntime::Model::Model(onnx::ModelProto&&, const PathString&, const IOnnxRuntimeOpSchemaRegistryList*, const onnxruntime::logging::Logger&, const onnxruntime::ModelOptions&) Unsupported model IR version: 9, max supported IR version: 8


2023-11-14 17:40:58.429095313 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 1232173, index: 1, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-11-14 17:40:58.429141433 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 1232175, index: 3, mask: {2, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-11-14 17:40:58.429134943 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 1232174, index: 2, mask: {65, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2023-11-14 17:40:58.429232271 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 1232176, index: 4, mask: {66, }, error code: 22 error msg: Invalid argument. Specify the

[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[W] Inference failed. You may want to try enabling partitioning to see better results. Note: Error was:
[ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Failed to load model with error: /onnxruntime_src/onnxruntime/core/graph/model.cc:146 onnxruntime::Model::Model(onnx::ModelProto&&, const PathString&, const IOnnxRuntimeOpSchemaRegistryList*, const onnxruntime::logging::Logger&, const onnxruntime::ModelOptions&) Unsupported model IR version: 9, max supported IR version: 8
UNetXL: fold constants
UNetXL: shape inference
UNetXL: finished
Building TensorRT engine for /nfs/students/rachwan/huggingface/.cache/onnx/clip.opt/model.onnx: /nfs/students/rachwan/huggingface/.cache/engine/clip.plan
Updating network outputs to ['text_embeddings', 'hidden_states']
Building TensorRT engine for /nfs/students/rachwan/huggingface/.cache/onnx/clip2.opt/model

[libprotobuf WARNING google/protobuf/io/coded_stream.cc:604] Reading dangerously large protocol message.  If the message turns out to be larger than 2147483647 bytes, parsing will be halted for security reasons.  To increase the limit (or to disable these warnings), see CodedInputStream::SetTotalBytesLimit() in google/protobuf/io/coded_stream.h.
[libprotobuf WARNING google/protobuf/io/coded_stream.cc:81] The total number of bytes read was 1390633693
[libprotobuf WARNING google/protobuf/io/coded_stream.cc:604] Reading dangerously large protocol message.  If the message turns out to be larger than 2147483647 bytes, parsing will be halted for security reasons.  To increase the limit (or to disable these warnings), see CodedInputStream::SetTotalBytesLimit() in google/protobuf/io/coded_stream.h.
[libprotobuf WARNING google/protobuf/io/coded_stream.cc:81] The total number of bytes read was 1390633693


Updating network outputs to ['text_embeddings', 'hidden_states']
Building TensorRT engine for /nfs/students/rachwan/huggingface/.cache/onnx/unetxl.opt/model.onnx: /nfs/students/rachwan/huggingface/.cache/engine/unetxl.plan
Loading TensorRT engine: /nfs/students/rachwan/huggingface/.cache/engine/clip.plan
Loading TensorRT engine: /nfs/students/rachwan/huggingface/.cache/engine/clip2.plan
Loading TensorRT engine: /nfs/students/rachwan/huggingface/.cache/engine/unetxl.plan
Success.
Error logging request, retrying in 3 seconds...


## Base Model Generation

In [4]:
%%time
pipe(prompt=prompt).images[0].show()

  0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 9.6 s, sys: 229 ms, total: 9.83 s
Wall time: 10.6 s


## Smashed Model Generation

In [5]:
%%time
smashed_model(prompt, image_height=1024, image_width=1024)[0]

CPU times: user 2.68 s, sys: 2.49 s, total: 5.17 s
Wall time: 5.22 s


[<PIL.Image.Image image mode=RGB size=1024x1024>]